In [ ]:
import os
import re

from importlib import reload

import numpy as np
import pandas as pd

from scipy.stats import norm
from scipy import stats

import seaborn as sns
import matplotlib.pylab as plt

from luescher_nd.database import utilities as ut
from luescher_nd.database.utilities import DATA_FOLDER

sns.set(context="notebook", style="ticks", font_scale=1)

In [ ]:
%load_ext blackcellmagic

In [ ]:
files = [f for f in os.listdir(DATA_FOLDER) if f.endswith(".sqlite") and not "tmp" in f]
print("\n".join([f"{n:2d} {f}" for n, f in enumerate(files)]))

In [ ]:
files = [f for f in files if "spherical" in f and "a1g" in f and "0.0" in f]

cols = ["L", "n1d", "nstep", "x", "contact_strength", "nlevel"]

dfs = []
for f in files:
    df = ut.read_table(os.path.join(DATA_FOLDER, f), filter_degeneracy=True)[cols]
    df["precise"] = False if "less-prec" in f else True
    dfs.append(df)


df = (
    pd.concat(dfs, ignore_index=True)
    .set_index(["precise", "L", "n1d", "nstep", "nlevel"])
    .sort_index()
    
)

diff = (df.loc[True] - df.loc[False]).dropna()

diff.head()


In [ ]:
fig, ax = plt.subplots(figsize=(2, 3), dpi=250)
counts = diff.reset_index().groupby(["L", "n1d", "nstep"])[["x"]].count()

y = 4 * np.arange(len(counts))
ax.barh(y, counts.values.flatten(), height=4)


ax.set_xlabel("count")

ax.set_yticks(y)
ax.set_yticklabels(["$%1.0f$, $%3d$, %3d" % el for el in counts.index], fontsize=4)
ax.set_ylabel("$L$, $n_{1d}$, $n_\mathrm{step}$")

sns.despine()

ax.set_title("Count of $x$-values")

plt.show(fig)

# Distribution of $x$-values

In [ ]:
grid = sns.FacetGrid(
    diff.reset_index(),
    row="n1d",
    col="L",
    hue="nstep",
    sharex="col",
    sharey=False,
    margin_titles=True,
)

grid.map(sns.distplot, "x", norm_hist=True)
grid.add_legend()
grid.set_xlabels("$\Delta x$")

sns.despine(grid.fig, left=True, trim=True)
for ax in grid.axes.flatten():
    ax.set_yticks([])

plt.show(grid.fig)

In [ ]:
grid = sns.FacetGrid(diff.reset_index(), col="L", row="n1d", sharex="col", sharey=False)

grid.map(sns.distplot, "x", norm_hist=True)

grid.add_legend()
grid.set_xlabels("$\Delta x$")

sns.despine(grid.fig, left=True, trim=True)
for ax in grid.axes.flatten():
    ax.set_yticks([])

plt.show(grid.fig)

In [ ]:
grid = sns.FacetGrid(diff.reset_index(), col="L", sharex=False, sharey=False)

grid.map(sns.distplot, "x", norm_hist=True)
grid.add_legend()
grid.set_xlabels("$\Delta x$")

sns.despine(grid.fig, left=True, trim=True)
for ax in grid.axes.flatten():
    ax.set_yticks([])

plt.show(grid.fig)

In [ ]:
vals = diff.x.dropna().sort_values()

vals = vals[(np.abs(stats.zscore(vals)) < 2)]


mu, std = norm.fit(vals)
x = np.linspace(diff.x.min(), diff.x.max(), 1000)
p = norm.pdf(x, mu, std)

In [ ]:
grid = sns.FacetGrid(diff.reset_index(), sharex=False, sharey=False, xlim=(-1.e-12, 1.e-12))

grid.map(sns.distplot, "x", norm_hist=True)

grid.map(sns.distplot, "x", norm_hist=True)
grid.add_legend()
grid.set_xlabels("$\Delta x$")

sns.despine(grid.fig, left=True, trim=True)
for ax in grid.axes.flatten():
    ax.set_yticks([])

ax.plot(x, p, label=f"$N(\mu={mu:1.2e}, \sigma={std:1.2e})$", color="green")
ax.legend(frameon=False, fontsize=6, loc="upper left", bbox_to_anchor=(0.6, 0.5))

grid.fig.set_dpi(250)

plt.show(grid.fig)

# Contact interaction

In [ ]:
cdiff = diff.reset_index().groupby(["L", "n1d", "nstep"])[["contact_strength"]].mean()

vals = cdiff.contact_strength.dropna().sort_values()

vals = vals[(np.abs(stats.zscore(vals)) < 4)]


mu, std = norm.fit(vals)
x = np.linspace(vals.min(), vals.max(), 1000)
p = norm.pdf(x, mu, std)

In [ ]:
grid = sns.FacetGrid(cdiff.reset_index(), sharex=False, sharey=False, xlim=(-5.e-15, 5.e-15))

grid.map(sns.distplot, "contact_strength", norm_hist=True)

grid.map(sns.distplot, "contact_strength", norm_hist=True)
grid.add_legend()
grid.set_xlabels("$\Delta c$")

sns.despine(grid.fig, left=True, trim=True)
for ax in grid.axes.flatten():
    ax.set_yticks([])

ax.plot(x, p, label=f"$N(\mu={mu:1.2e}, \sigma={std:1.2e})$", color="green")
ax.legend(frameon=False, fontsize=6, loc="upper left", bbox_to_anchor=(0.6, 0.5))

grid.fig.set_dpi(250)

plt.show(grid.fig)